In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cosine
import sklearn
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['movie.csv', 'tag.csv', 'genome_tags.csv', 'genome_scores.csv', 'link.csv', 'rating.csv']


In [2]:
ratings = pd.read_csv("../input/rating.csv")
movies = pd.read_csv("../input/movie.csv")

movies= movies.loc[:,["movieId","title"]]
#print(movies.head(10)) #27277

ratings = ratings.loc[:,["userId","movieId","rating"]]
#print(ratings.head(10))

# then merge movie and rating data
data = pd.merge(movies,ratings)
#print(data.userId.unique().shape[0]) #138493
#print(data.title.unique().shape[0]) #26729
#print(data.movieId.unique().shape[0]) #26744

data.shape
data = data.iloc[:3000000,:]
#print(data.userId.unique().shape[0]) #137065
#print(data.movieId.unique().shape[0]) #886

In [3]:
pivot_table = pd.pivot_table(data, index = ["userId"],columns = ["title"],values = "rating", fill_value=0, dropna=True)
print(pivot_table)

title   8 Seconds (1994)                  ...                    Young Poisoner's Handbook, The (1995)
userId                                    ...                                                         
1                    0.0                  ...                                                      0.0
2                    0.0                  ...                                                      0.0
3                    0.0                  ...                                                      0.0
4                    0.0                  ...                                                      0.0
5                    0.0                  ...                                                      0.0
6                    0.0                  ...                                                      0.0
7                    0.0                  ...                                                      0.0
8                    0.0                  ...                            

In [4]:
def findksimilarusers(user_id, pivot_ratings, k=50):
    similarities=[]
    indices=[]
    model_knn = NearestNeighbors(metric = cosine, algorithm = 'brute') 
    model_knn.fit(pivot_ratings)

    distances, indices = model_knn.kneighbors(pivot_ratings.loc[user_id].values.reshape(1, -1), n_neighbors = k+1)
    print(distances)
    print(indices)
    print(indices.shape)
    print(indices.flatten().shape)
    similarities = 1-distances.flatten()
    print(similarities)
    print(indices.flatten())
    print(similarities.shape)
    print(len(indices.flatten()))
    print(indices.flatten()[1])
    print ('{0} most similar users for User {1}:\n'.format(k,user_id))
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i] == user_id:
            continue;

        else:
            print ('{0}: User {1}, with similarity of {2}'.format(i, indices.flatten()[i], similarities[i]))
            
    return similarities,indices

similarities,indices = findksimilarusers(4,pivot_table)

[[0.         0.36800186 0.38756743 0.40047451 0.40643015 0.41249403
  0.41381874 0.41649572 0.41974779 0.42338861 0.42572805 0.43037059
  0.43097863 0.43280019 0.43613531 0.4388272  0.4408312  0.44221413
  0.44848184 0.44874695 0.44977768 0.45063169 0.45138101 0.45151604
  0.45279513 0.45349282 0.45490436 0.45503395 0.45505694 0.45507444
  0.45617172 0.45726783 0.4573326  0.4591708  0.45920013 0.46010193
  0.46046671 0.4606621  0.46143388 0.462135   0.46258998 0.46297191
  0.46377291 0.46457231 0.46612958 0.46736436 0.46864136 0.46877605
  0.46890653 0.46919571 0.46923964]]
[[     3  38485  32139 109741 103101  31322   9720  89052   8068  37859
  126106   8538 131932   6679  50456  42466  37182  19012 110015  26473
   16966  92254  75275  25927  60142  76606  30463 121428  15458  71046
    2388  42213  78417  73695  83199  15278 109317 125582  77959  40976
   54775  19067  10664  20739  89472  58015  86486 117299  55997  53713
   81817]]
(1, 51)
(51,)
[1.         0.63199814 0.61243257 

In [5]:
def recommendations(user_id, similarities, indices):
    actual_indices = [x for x in indices.flatten()]
#    print(actual_indices)
    sim_ind = dict(zip(actual_indices, similarities))
    #print(sim_ind)

    del sim_ind[user_id-1]
    print(sim_ind)
    max_ind = [key for m in [max(sim_ind.values())] for key,val in sim_ind.items() if val == m]
    #print(max_ind)
    
    for maxSim_row in max_ind:
        similarUser = pivot_table.loc[maxSim_row,:]
        currUser = pivot_table.loc[user_id,:]   
#        print(currUser)
#        print(similarUser)
        currUser_zeros = currUser[currUser==0]
        print(currUser_zeros)
        for k in currUser_zeros.keys():
            if similarUser[k] != 0:
                print('Recommendations for User {0}: {1}'.format(user_id, k))
recommendations(4, similarities, indices)

{38485: 0.6319981359672576, 32139: 0.6124325692277502, 109741: 0.5995254864480521, 103101: 0.5935698536180276, 31322: 0.5875059683188942, 9720: 0.5861812632097712, 89052: 0.5835042805286678, 8068: 0.580252207947732, 37859: 0.5766113899158557, 126106: 0.5742719495383168, 8538: 0.5696294138810952, 131932: 0.5690213737508669, 6679: 0.5671998137095519, 50456: 0.5638646887101163, 42466: 0.5611728030784412, 37182: 0.5591687964413298, 19012: 0.5577858716092933, 110015: 0.5515181624257245, 26473: 0.5512530485000962, 16966: 0.5502223233113924, 92254: 0.5493683099522703, 75275: 0.5486189943693173, 25927: 0.5484839634734997, 60142: 0.547204871847462, 76606: 0.5465071808037829, 30463: 0.5450956371187881, 121428: 0.5449660504752176, 15458: 0.5449430633779707, 71046: 0.5449255609198993, 2388: 0.5438282793983183, 42213: 0.5427321675527594, 78417: 0.5426674017863917, 73695: 0.5408291994068049, 83199: 0.5407998690797751, 15278: 0.5398980684968568, 109317: 0.539533291201244, 125582: 0.539337897914422, 7

In [6]:
x = (pivot_table.loc[4]).values.reshape(1,-1)
y = (pivot_table.loc[38500]).values.reshape(1,-1)
print(x.shape)
print(y.shape)
print(sklearn.metrics.pairwise.cosine_similarity(x,y))

(1, 438)
(1, 438)
[[0.17897495]]
